## 작업형3

### 문제1. 주어진 조개 데이터 300개 중 앞에서부터 210개는 train 데이터로 만들고, 나머지 90개는 test데이터로 만든다. 모델을 학습(적합)할 때는 train데이터를 사용하고, 예측할 때는 test데이터를 사용한다. 모델은 로지스틱 회귀를 써서 성별(gender)을 예측하되, 패널티는 부과하지 않는다.
- 종속변수: gender(0:암컷, 1:수컷)

### 문제1-1. weight를 독립변수로 gender를 종속변수로 사용하여 로지스틱 회귀 모형을 만들고, weight 변수가 한 단위 증가할 때 수컷일 오즈비 값은? (반올림하여 소수 넷째자리까지 계산)

In [4]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/refs/heads/main/p4/7_3/clam.csv")

# 데이터셋 분할
train = df.iloc[:210]
test = df.iloc[210:]
print(train.shape, test.shape)

(210, 6) (90, 6)


In [8]:
from statsmodels.formula.api import logit
import numpy as np

# 로지스틱 회귀 모델 생성 및 학습
model = logit("gender ~ weight", data=train).fit()
print(model.summary())
# 오즈비 계산
print(round(np.exp(model.params['weight']), 4))

Optimization terminated successfully.
         Current function value: 0.690045
         Iterations 4
                           Logit Regression Results                           
Dep. Variable:                 gender   No. Observations:                  210
Model:                          Logit   Df Residuals:                      208
Method:                           MLE   Df Model:                            1
Date:                Thu, 21 Nov 2024   Pseudo R-squ.:                0.003431
Time:                        23:48:20   Log-Likelihood:                -144.91
converged:                       True   LL-Null:                       -145.41
Covariance Type:            nonrobust   LLR p-value:                    0.3178
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.3140      0.276     -1.137      0.256      -0.855       0.227
weight         0.0047      0.

### 문제1-2. gender를 종속변수로 하고 나머지 변수들(age, length, diameter, height, weight)을 독립변수로 사용하는 로지스틱 회귀 모델을 적합시킨 후, 잔차 이탈도(Residual Deviance)를 계산하시오. (반올림하여 소수 둘째자리까지 계산)

In [12]:
from statsmodels.formula.api import glm
import statsmodels.api as sm

# glm 모델 적합 (로지스틱 회귀를 위해 이항 분포 사용)
formula = "gender ~ age + length + diameter + height + weight"
model = glm(formula, data=train, family=sm.families.Binomial()).fit()
# 잔차이탈도 계산
print(model.summary())
print(round(model.deviance, 2))

                 Generalized Linear Model Regression Results                  
Dep. Variable:                 gender   No. Observations:                  210
Model:                            GLM   Df Residuals:                      204
Model Family:                Binomial   Df Model:                            5
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -143.47
Date:                Thu, 21 Nov 2024   Deviance:                       286.93
Time:                        23:52:49   Pearson chi2:                     210.
No. Iterations:                     4   Pseudo R-squ. (CS):            0.01833
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.6701      0.651      1.030      0.3

### 문제1-3. 독립변수 weight 만 사용해 학습한 로지스틱 회귀모델에서 test데이터의 gender를 예측하고, error rate(오류율)를 구하시오. (반올림하여 소수 셋째자리까지 계산)

In [17]:
from sklearn.metrics import accuracy_score
from statsmodels.formula.api import logit
train = df.iloc[:210]
test = df.iloc[210:]

# 학습 test데이터를 사용해 예측(0.5미만: 0, 0.5이상: 1)
model = logit("gender ~ weight", data=train).fit()
target = test.pop('gender')
pred = model.predict(test) > 0.5
# 실제 값과 예측값을 사용하여 정확도 계산
acc = accuracy_score(target, pred)
# 오류율 계산
print(round(1-acc, 3))

Optimization terminated successfully.
         Current function value: 0.690045
         Iterations 4
0.478


### 문제2-1. ERP와 가장 상관관계 높은값을 구하시오. (반올림하여 소수 셋째자리까지 계산)

In [19]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/refs/heads/main/p4/7_3/system.csv")

In [28]:
df.head()

# ERP와 각 변수 사이의 상관계수 계산
id = df.corr()['ERP'][1:].abs().idxmax()
df.corr()['ERP'][1:][id]

0.43444249747725106

### 문제2-2. CPU 컬럼이 100미만인 것만 찾아 ERP를 종속 변수로, 나머지 변수들을 독립 변수로 설정해 선형회귀 모델을 만들고 적합한 결정계수를 구하시오. (반올림하여 소수 셋째자리까지 계산)

In [38]:
from statsmodels.formula.api import ols

# CPU가 100 미만인 데이터 필터링
cond = df['CPU']<100
df=df[cond]

# 선형회귀 모델 생성: ERP를 종속변수로, 나머지 변수들을 독립 변수로 설정
formula = "ERP~Feature1+Feature2+Feature3+CPU"
model = ols(formula, data=df).fit()

# 모델 요약 정보 출력
print(model.summary())
print(round(model.rsquared, 3))

                            OLS Regression Results                            
Dep. Variable:                    ERP   R-squared:                       0.226
Model:                            OLS   Adj. R-squared:                  0.193
Method:                 Least Squares   F-statistic:                     6.937
Date:                Fri, 22 Nov 2024   Prob (F-statistic):           6.07e-05
Time:                        00:20:43   Log-Likelihood:                -429.44
No. Observations:                 100   AIC:                             868.9
Df Residuals:                      95   BIC:                             881.9
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     74.4348     15.094      4.932      0.0

### 문제 2-3. 2-2에서 만든 모델에서 독립변수 중 p-value가 가장 높은값을 구하시오. (반올림하여 소수 셋째자리까지 계산)

In [37]:
round(model.pvalues.max(), 3)

0.457